<a href="https://colab.research.google.com/github/MatCat776/InterviewBot/blob/main/interview_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My Job Interview Bot


## Configuration

### Install Libraries

In [ ]:
%pip install --upgrade --quiet google-genai pandas
# not sure I really need pandas


In [ ]:
import base64

### Configure Gemini

In [ ]:
from google import genai
from google.genai import types
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('AI_STUDIO_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)
model = "gemini-2.5-flash-lite"


### Configure google drive

In [ ]:
    from google.colab import drive
    drive.mount('/content/drive')

## Static model settings

In [ ]:
#TODO: move resume import here
my_instructions = """
You are a job applicant in a job interview.
"""

generate_content_config = types.GenerateContentConfig(
    temperature = .5,
    top_p = 0.95,
    max_output_tokens = 2048,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=my_instructions)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=0,
    ),
  )

## Interview Questions

In [ ]:
contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Please introduce yourself""")
      ]
    )
  ]

## Answer Questions

In [ ]:
for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    print(chunk.text, end="")

## Questions that use my resume, text method

In [ ]:
# Read the content of the text file
with open("/content/drive/My Drive/text_resume.txt", "r") as f:
    text_resume_content = f.read()

contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Tell me about the document"""),
        types.Part.from_text(text=text_resume_content), # Include the text content directly
        types.Part.from_text(text="""Say Boo when done""")
      ]
    )
  ]

# # Response as a stream
#for chunk in client.models.generate_content_stream(
#    model = model,
#    contents = contents,
#    config = generate_content_config,
#    ):
#    print(chunk.text, end="")

# Respond as a single answer
response = client.models.generate_content(
    model=model, contents=contents, config = generate_content_config,
)

print(response.text)

## Questions that use my resume, PDF method.
Pulling content from google drive and github doesn't seem to work in colab the same way it does in Virtex AI. For some reason, I have to convert the pdf to txt, then I can use it here.

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber

def convert_pdf_to_txt_pdfplumber(pdf_path, txt_path):
    """Converts a PDF file to a plain text file using pdfplumber."""
    try:
        with pdfplumber.open(pdf_path) as pdf, open(txt_path, "w", encoding="utf-8") as f:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    f.write(text + '\n')  # Add a newline to separate pages
        print(f"Text successfully extracted from {pdf_path} to {txt_path}")
    except Exception as e:
        print(f"Error converting PDF: {e}")

# Example usage:
# convert_pdf_to_txt_pdfplumber("input.pdf", "output.txt")

In [ ]:
convert_pdf_to_txt_pdfplumber("/content/drive/My Drive/My_Resume.pdf", "/content/drive/My Drive/resume_as_txt.txt")

with open("/content/drive/My Drive/resume_as_txt.txt", "r") as f:
    text_resume_content = f.read()

contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Tell me about the document"""),
        types.Part.from_text(text=text_resume_content), # Include the text content directly
        types.Part.from_text(text="""Say Boo PDF when done""")
      ]
    )
  ]



for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    print(chunk.text, end="")